# Lesson 3: Agentic Search

In [1]:
# libraries
from dotenv import load_dotenv
import os
from tavily import TavilyClient

# load environment variables from .env file
_ = load_dotenv()

# connect
client = TavilyClient(api_key=os.environ.get("TAVILY_API_KEY"))

In [2]:
# run search
result = client.search("What is in Nvidia's new Blackwell GPU?",
                       include_answer=True)

# print the answer
result["answer"]


"The new Blackwell GPU features two dies with 192GB to 288GB of HBM3e memory and a 10 TB/s interconnect. It includes a second-generation Transformer Engine and advanced NVLink. It's designed for generative AI and large-scale AI models."

## Regular search

In [3]:
# choose location (try to change to your own city!)

city = "Amsterdam"

query = f"""
    what is the current weather in {city}?
    Should I travel there today?
    "weather.com"
"""

> Note: search was modified to return expected results in the event of an exception. High volumes of student traffic sometimes cause rate limit exceptions.

In [4]:
import requests
from bs4 import BeautifulSoup
from duckduckgo_search import DDGS
import re

ddg = DDGS()

def search(query, max_results=6):
    try:
        results = ddg.text(query, max_results=max_results)
        return [i["href"] for i in results]
    except Exception as e:
        print(f"returning previous results due to exception reaching ddg.")
        results = [ # cover case where DDG rate limits due to high deeplearning.ai volume
            "https://weather.com/weather/today/l/USCA0987:1:US",
            "https://weather.com/weather/hourbyhour/l/54f9d8baac32496f6b5497b4bf7a277c3e2e6cc5625de69680e6169e7e38e9a8",
        ]
        return results  


for i in search(query):
    print(i)

https://www.zhihu.com/question/19565664
https://www.zhihu.com/question/20831164
https://answers.microsoft.com/en-us/msteams/forum/all/ms-teams-is-always-asking-for-my-password/3e15e820-9fce-4d32-b734-6ee770668ede
https://answers.microsoft.com/en-us/outlook_com/forum/all/using-rules-to-delete-items-from-deleted-items/9e229952-77bb-4e5d-a132-a7599072e90b
https://answers.microsoft.com/en-us/outlook_com/forum/all/outlook-365-rules-not-running-automatically/1b18f3e6-6427-4291-aa08-56855a9e7415
https://answers.microsoft.com/en-us/outlook_com/forum/all/hotmail-login-constantly-asking-to-reset-password/680ad7e7-2e89-499b-967a-e5e39ac5d3e9


/var/folders/r2/n1vtxc157gz88z42zprlblb00000gn/T/ipykernel_10750/2852339428.py:6: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  ddg = DDGS()


In [5]:
def scrape_weather_info(url):
    """Scrape content from the given URL"""
    if not url:
        return "Weather information could not be found."
    
    # fetch data
    headers = {'User-Agent': 'Mozilla/5.0'}
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        return "Failed to retrieve the webpage."

    # parse result
    soup = BeautifulSoup(response.text, 'html.parser')
    return soup


> Note: This produces a long output, you may want to right click and clear the cell output after you look at it briefly to avoid scrolling past it.

In [6]:
# use DuckDuckGo to find websites and take the first result
url = search(query)[0]

# scrape first wesbsite
soup = scrape_weather_info(url)

print(f"Website: {url}\n\n")
print(str(soup.body)[:50000]) # limit long outputs

Website: https://colab.research.google.com/


<body class=""><div class="onegoogle"><div class="gb_Fa gb_Kd gb_4d" id="gb"><div class="gb_Dd gb_1d gb_yd" data-ogsr-up="" ng-non-bindable="" style="padding:0"><div class="gb_Te" style="display:block"><div class="gb_4c"></div><a aria-label="Sign in" class="gb_Ua gb_Ad gb_rd gb_id" href="https://accounts.google.com/ServiceLogin?passive=true&amp;continue=https%3A%2F%2Fcolab.research.google.com%2F&amp;ec=GAZAqQM" target="_top"><span class="gb_Vd">Sign in</span></a></div></div></div><script nonce="lecnLvqLrizGMFApEv2zdQ==">this.gbar_=this.gbar_||{};(function(_){var window=this;
try{
_.ke=function(a,b,c){if(!a.j)if(c instanceof Array){c=_.C(c);for(var d=c.next();!d.done;d=c.next())_.ke(a,b,d.value)}else{d=(0,_.D)(a.C,a,b);var e=a.v+c;a.v++;b.dataset.eqid=e;a.B[e]=d;b&&b.addEventListener?b.addEventListener(c,d,!1):b&&b.attachEvent?b.attachEvent("on"+c,d):a.o.log(Error("G`"+b))}};
}catch(e){_._DumpException(e)}
try{
_.le=function(){if(!_.t.addEve

In [7]:
# extract text
weather_data = []
for tag in soup.find_all(['h1', 'h2', 'h3', 'p']):
    text = tag.get_text(" ", strip=True)
    weather_data.append(text)

# combine all elements into a single string
weather_data = "\n".join(weather_data)

# remove all spaces from the combined text
weather_data = re.sub(r'\s+', ' ', weather_data)
    
print(f"Website: {url}\n\n")
print(weather_data)

Website: https://colab.research.google.com/





## Agentic Search

In [8]:
# run search
result = client.search(query, max_results=1)

# print first result
data = result["results"][0]["content"]

print(data)

{'location': {'name': 'Amsterdam', 'region': 'North Holland', 'country': 'Netherlands', 'lat': 52.374, 'lon': 4.8897, 'tz_id': 'Europe/Amsterdam', 'localtime_epoch': 1755251224, 'localtime': '2025-08-15 11:47'}, 'current': {'last_updated_epoch': 1755251100, 'last_updated': '2025-08-15 11:45', 'temp_c': 26.2, 'temp_f': 79.2, 'is_day': 1, 'condition': {'text': 'Sunny', 'icon': '//cdn.weatherapi.com/weather/64x64/day/113.png', 'code': 1000}, 'wind_mph': 7.4, 'wind_kph': 11.9, 'wind_degree': 295, 'wind_dir': 'WNW', 'pressure_mb': 1023.0, 'pressure_in': 30.21, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 61, 'cloud': 0, 'feelslike_c': 26.7, 'feelslike_f': 80.0, 'windchill_c': 26.1, 'windchill_f': 79.1, 'heatindex_c': 26.6, 'heatindex_f': 80.0, 'dewpoint_c': 14.2, 'dewpoint_f': 57.6, 'vis_km': 10.0, 'vis_miles': 6.0, 'uv': 3.4, 'gust_mph': 8.5, 'gust_kph': 13.7}}


In [9]:
import json
from pygments import highlight, lexers, formatters

# parse JSON
parsed_json = json.loads(data.replace("'", '"'))

# pretty print JSON with syntax highlighting
formatted_json = json.dumps(parsed_json, indent=4)
colorful_json = highlight(formatted_json,
                          lexers.JsonLexer(),
                          formatters.TerminalFormatter())

print(colorful_json)


{
    "location": {
        "name": "Amsterdam",
        "region": "North Holland",
        "country": "Netherlands",
        "lat": 52.374,
        "lon": 4.8897,
        "tz_id": "Europe/Amsterdam",
        "localtime_epoch": 1755251224,
        "localtime": "2025-08-15 11:47"
    },
    "current": {
        "last_updated_epoch": 1755251100,
        "last_updated": "2025-08-15 11:45",
        "temp_c": 26.2,
        "temp_f": 79.2,
        "is_day": 1,
        "condition": {
            "text": "Sunny",
            "icon": "//cdn.weatherapi.com/weather/64x64/day/113.png",
            "code": 1000
        },
        "wind_mph": 7.4,
        "wind_kph": 11.9,
        "wind_degree": 295,
        "wind_dir": "WNW",
        "pressure_mb": 1023.0,
        "pressure_in": 30.21,
        "precip_mm": 0.0,
        "precip_in": 0.0,
        "humidity": 61,
        "cloud": 0,
        "feelslike_c": 26.7,
        "feelslike_f": 80.0,
        "windchill_c": 26.1,
        "windchill_f": 79.1,
    

<img src="./google_sample.png" width="800" height="600">